In [2]:
import numpy as np
import os
import re
import ast
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

# === CONFIGURATION ===
base_path = "data/"
inputs_txt = "week3/inputs.txt"   # file dari email minggu lalu
outputs_txt = "week3/outputs.txt"
n_candidates = 2000

# === 1. LOAD INPUTS ===
def load_inputs(path):
    with open(path, "r") as f:
        text = f.read().strip()

    # Pecah tiap array(...) jadi satu per function
    chunks = re.findall(r'array\((.*?)\)', text, re.DOTALL)

    inputs = []
    for ch in chunks:
        # Bersihkan karakter aneh dan pecah angka
        numbers = [float(x) for x in re.findall(r'[-+]?\d*\.\d+|\d+', ch)]
        inputs.append(np.array(numbers))
    return inputs

# === 2. LOAD OUTPUTS ===
def load_outputs(path):
    with open(path, "r") as f:
        text = f.read().strip()

    # Ambil angka dari np.float64(...)
    numbers = [float(x) for x in re.findall(r'[-+]?\d*\.\d+|\d+', text)]
    return np.array(numbers)

# === LOAD FROM EMAIL ===
new_inputs = load_inputs(inputs_txt)
new_outputs = load_outputs(outputs_txt)

print(f"✅ Loaded {len(new_inputs)} input vectors and {len(new_outputs)} outputs.")

# === 3. PROCESS EACH FUNCTION ===
queries_out = []

for i in range(1, 9):
    folder = os.path.join(base_path, f"function_{i}")
    input_file = os.path.join(folder, "week3_inputs.npy")
    output_file = os.path.join(folder, "week3_outputs.npy")

    # Load minggu lalu
    X_prev = np.load(input_file)
    y_prev = np.load(output_file)

    # Gabungkan dengan hasil minggu lalu
    X_combined = np.vstack([X_prev, new_inputs[i - 1].reshape(1, -1)])
    y_combined = np.append(y_prev, new_outputs[i - 1])

    # Simpan versi baru
    np.save(os.path.join(folder, "week4_inputs.npy"), X_combined)
    np.save(os.path.join(folder, "week4_outputs.npy"), y_combined)

    dim = X_combined.shape[1]
    print(f"\n=== Function {i} ({dim}D) ===")
    print(f"  Data size: {len(X_combined)} points, y range: [{y_combined.min():.4f}, {y_combined.max():.4f}]")

    # === TRAIN MLP MODEL ===
    model = make_pipeline(
        StandardScaler(),
        MLPRegressor(
            hidden_layer_sizes=(64, 32),
            activation='relu',
            solver='adam',
            learning_rate_init=0.01,
            max_iter=2000,
            random_state=42
        )
    )
    model.fit(X_combined, y_combined)

    # === GENERATE NEW QUERY ===
    candidates = np.random.uniform(0, 1, (n_candidates, dim))
    preds = model.predict(candidates)
    best_idx = np.argmax(preds)
    best_query = candidates[best_idx]

    query_str = "-".join([f"{x:.6f}" for x in best_query])
    print(f"  Best predicted output: {preds[best_idx]:.6f}")
    print(f"  Query to submit: {query_str}")

    queries_out.append(f"Function {i}: {query_str}")

# === 4. SAVE QUERIES ===
with open("week4_queries.txt", "w") as f:
    f.write("\n".join(queries_out))

print("\n💾 All week 4 queries saved to week4_queries.txt")
print("✅ DONE — Copy each query into the Capstone portal.")

✅ Loaded 24 input vectors and 51 outputs.

=== Function 1 (2D) ===
  Data size: 13 points, y range: [-0.0036, 64.0000]
  Best predicted output: 40.772821
  Query to submit: 0.016007-0.605447

=== Function 2 (2D) ===
  Data size: 13 points, y range: [-0.0656, 3.1124]
  Best predicted output: 1.707927
  Query to submit: 0.728518-0.696110

=== Function 3 (3D) ===
  Data size: 18 points, y range: [-0.3989, 71.0000]
  Best predicted output: 59.816766
  Query to submit: 0.049613-0.978013-0.976174

=== Function 4 (4D) ===
  Data size: 33 points, y range: [-32.6257, 64.0000]
  Best predicted output: 24.649696
  Query to submit: 0.248046-0.305402-0.210307-0.342861

=== Function 5 (4D) ===
  Data size: 23 points, y range: [0.1129, 1787.3908]
  Best predicted output: 2166.909172
  Query to submit: 0.022283-0.065706-0.891871-0.948172

=== Function 6 (5D) ===
  Data size: 23 points, y range: [-2.5712, 64.0000]
  Best predicted output: 62.366539
  Query to submit: 0.019569-0.199363-0.004800-0.829687